In [ ]:
import numpy as np
from keras.models import Model, load_model
from keras.layers import Dense,BatchNormalization,Conv1D
from keras.layers import Input,GlobalMaxPooling1D, GlobalAveragePooling1D, concatenate
from keras.optimizers import Adam
from utils import generate, generate_sim, OrnsteinUng
import tensorflow as tf
from keras.callbacks import EarlyStopping,ReduceLROnPlateau,ModelCheckpoint,CSVLogger
import datetime
from keras.callbacks import History 
import matplotlib.pyplot as plt
import pickle
import glob
import os

In [ ]:
history = History()

batchsize = 32
T = np.arange(19,21,0.1) # this provides another layer of stochasticity to make the network more robust
steps = 20
 # number of steps to generate
initializer = 'he_normal'
f = 32 #number of filters
sigma = 0 #noise variance

inputs = Input((steps-1,1))

x1 = Conv1D(f,2,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,2,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = Conv1D(f,2,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x1)
x1 = BatchNormalization()(x1)
x1 = GlobalMaxPooling1D()(x1)


x2 = Conv1D(f,2,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x2 = BatchNormalization()(x2)
x2 = Conv1D(f,2,dilation_rate=3,padding='causal',activation='relu',kernel_initializer=initializer)(x2)
x2 = BatchNormalization()(x2)
x2 = Conv1D(f,2,dilation_rate=5,padding='causal',activation='relu',kernel_initializer=initializer)(x2)
x2 = BatchNormalization()(x2)
x2 = GlobalMaxPooling1D()(x2)


x3 = Conv1D(f,3,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x3 = BatchNormalization()(x3)
x3 = Conv1D(f,3,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x3)
x3 = BatchNormalization()(x3)
x3 = Conv1D(f,3,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x3)
x3 = BatchNormalization()(x3)
x3 = GlobalMaxPooling1D()(x3)


x4 = Conv1D(f,3,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x4 = BatchNormalization()(x4)
x4 = Conv1D(f,3,dilation_rate=3,padding='causal',activation='relu',kernel_initializer=initializer)(x4)
x4 = BatchNormalization()(x4)
x4 = Conv1D(f,3,dilation_rate=5,padding='causal',activation='relu',kernel_initializer=initializer)(x4)
x4 = BatchNormalization()(x4)
x4 = GlobalMaxPooling1D()(x4)


x5 = Conv1D(f,4,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x5 = BatchNormalization()(x5)
x5 = Conv1D(f,4,dilation_rate=2,padding='causal',activation='relu',kernel_initializer=initializer)(x5)
x5 = BatchNormalization()(x5)
x5 = Conv1D(f,4,dilation_rate=4,padding='causal',activation='relu',kernel_initializer=initializer)(x5)
x5 = BatchNormalization()(x5)
x5 = GlobalMaxPooling1D()(x5)

x7 = Conv1D(f,4,padding='causal',activation='relu',kernel_initializer=initializer)(inputs)
x7 = BatchNormalization()(x7)
x7 = Conv1D(f,4,dilation_rate=3,padding='causal',activation='relu',kernel_initializer=initializer)(x7)
x7 = BatchNormalization()(x7)
x7 = Conv1D(f,4,dilation_rate=5,padding='causal',activation='relu',kernel_initializer=initializer)(x7)
x7 = BatchNormalization()(x7)
x7 = GlobalMaxPooling1D()(x7)


x6 = Conv1D(f,7,padding='same',activation='relu',kernel_initializer=initializer)(inputs)
x6 = BatchNormalization()(x6)
x6 = GlobalMaxPooling1D()(x6)


con = concatenate([x1,x2,x3,x4,x5,x6])
dense = Dense(512,activation='relu')(con)
# dense = Dense(256,activation='relu')(dense)
dense = Dense(128,activation='relu')(dense)
# dense = Dense(64,activation='relu')(dense)
# dense = Dense(32,activation='relu')(dense)
dense2 = Dense(3,activation='softmax')(dense)
model = Model(inputs=inputs, outputs=dense2)

optimizer = Adam(lr=1e-5)
model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['acc'])
model.summary()


callbacks = [
         ReduceLROnPlateau(monitor='val_loss',
                           factor=0.1,
                           patience=4,
                           verbose=1,
                           min_lr=1e-9),
         ModelCheckpoint(filepath=f'./Models/{steps}_model.h5',
                         monitor='val_acc',
                         save_best_only=False,
                         mode='max',
                         save_weights_only=False), history]


gen = generate(batchsize=batchsize,steps=steps,T=T,sigma=sigma)
model.fit_generator(generator=gen,
        steps_per_epoch=50,
        epochs=25,
        verbose=1,
        callbacks=callbacks,
        validation_data=generate(batchsize=batchsize,steps=steps,T=T,sigma=sigma),
        validation_steps=25)


train_loss = history.history['loss']
val_loss   = history.history['val_loss']
train_acc  = history.history['acc']
val_acc    = history.history['val_acc']
xc         = range(25)


##https://stackoverflow.com/questions/11026959/writing-a-dict-to-txt-file-and-reading-it-back
with open(f'maxpoolingtwodense/{steps}_history.txt', 'wb') as handle:
    pickle.dump(history.history, handle)

plt.figure()
plt.plot(xc, train_loss, label="train loss")
plt.plot(xc, val_loss, label="validation loss")
plt.legend()
plt.savefig(f"maxpoolingtwodense/{steps}_loss.png")

In [ ]:
plt.figure()
sizes = []
max_val_acc = []

dir_path = os.getcwd() + "added_layers"
matching_files = glob.glob( dir_path +"\*_history*")

for file in matching_files:
    with open(file, 'rb') as handle:
        history = pickle.loads(handle.read())
        train_loss = history['loss']
        val_loss   = history['val_loss']
        train_acc  = history['acc']
        val_acc    = history['val_acc']
        
        max_val_acc.append(max(val_acc))
    string = [s for s in file if s.isdigit()]
    actualstring = ""
    for s in string:
        actualstring += s
    sizes.append(int(actualstring))
zipped = zip(sizes, max_val_acc)
sorted_pairs = sorted(zipped)
sizes = [sorted_pairs[i][0] for i in range(len(sorted_pairs))]
max_val_acc = [sorted_pairs[i][1] for i in range(len(sorted_pairs))]

print(sorted_pairs)
print(sizes)
print(max_val_acc)
plt.xlabel("number of training steps")
plt.ylabel("validation accuracy")
plt.title("Training steps vs max validation accuracy")
plt.plot(sizes, max_val_acc)

#         plt.plot(xc, val_loss, label="validation loss")
#         plt.legend()

In [ ]:
def classification_on_sim(net_file, num_steps):
    dx,label=generate_sim(batchsize=100,steps=num_steps,T=15,sigma=0.00)
    ### change here to load a different network model
    print(dx)
    N=np.shape(dx)[0]
#     net_file = './Models/model_testsigma0-wB-6layern-300-variance-FBMCTRW.h5'
    model = load_model(net_file)
    predictions = []
    for j in range(N):
        dummy = np.zeros((1,num_steps-1,1))
        dummy[0,:,:] = dx[j,:,:]
        y_pred = model.predict(dummy) # get the results for 1D 
        print(y_pred)
        ymean = np.mean(y_pred,axis=0) # calculate mean prediction of N-dimensional trajectory 
        prediction = np.argmax(ymean,axis=0) # translate to classification
        predictions.append(prediction)

    accuracy = 100 * np.sum([1 if label[i] == predictions[i] else 0 for i in range(len(label))]) / len(label)
    return accuracy


In [ ]:
accuracies = []
# for i in range(10, 101, 10):
accuracies.append(classification_on_sim('./Models/300_model.h5', 300))
print(accuracies)

In [ ]:
plt.xlabel("Number of steps trained on")
plt.ylabel("test accuracy")
plt.title("Training steps vs Testing accuracy")
plt.plot(range(10, 101, 10),accuracies)
